## Analytic Programming

> Final: Analytic Programming, NTU, Spring, 2021.

Kuo, Yao-Jen <yaojenkuo@ntu.edu.tw> from [DATAINPOINT](https://www.datainpoint.com)

## Instructions

- We've imported necessary modules/libraries at the beginning of each exercise.
- We've put necessary files(if any) in the working directory of each exercise.
- We've defined the names of functions/inputs/arguments for you.
- Write down your solution between the comments `### BEGIN SOLUTION` and `### END SOLUTION`.
- Running tests to see if your solutions are right: Kernel -> Restart & Run All -> Restart and Run All Cells.
- You can run tests after each question or after finishing all questions.
- REMEMBER to upload your `.ipynb` file to [CEIBA](https://ceiba.ntu.edu.tw/) before **2021-06-18 20:59:59** when you are done running tests.

In [161]:
import json
import unittest
import numpy as np
import pandas as pd

## 00. Define a function named `extract_season_standings` that is able to extract the standings of NBA in 2020-2021 regular season given `standings_all.json` as a pandas DataFrame.

- Expected inputs: None.
- Expected outputs: a (30, 7) DataFrame.

```
        teamId  homeWin  homeLoss  awayWin  awayLoss  win  loss
0   1610612762       31         5       21        15   52    20
1   1610612756       27         9       24        12   51    21
2   1610612755       29         7       20        16   49    23
3   1610612751       28         8       20        16   48    24
4   1610612743       25        11       22        14   47    25
5   1610612746       26        10       21        15   47    25
6   1610612749       26        10       20        16   46    26
7   1610612742       21        15       21        15   42    30
8   1610612747       21        15       21        15   42    30
9   1610612757       20        16       22        14   42    30
10  1610612737       25        11       16        20   41    31
11  1610612752       25        11       16        20   41    31
12  1610612748       21        15       19        17   40    32
13  1610612744       25        11       14        22   39    33
14  1610612763       18        18       20        16   38    34
15  1610612738       21        15       15        21   36    36
16  1610612754       13        23       21        15   34    38
17  1610612764       19        17       15        21   34    38
18  1610612766       18        18       15        21   33    39
19  1610612759       14        22       19        17   33    39
20  1610612741       15        21       16        20   31    41
21  1610612740       18        18       13        23   31    41
22  1610612758       16        20       15        21   31    41
23  1610612761       16        20       11        25   27    45
24  1610612750       13        23       10        26   23    49
25  1610612739       13        23        9        27   22    50
26  1610612760       10        26       12        24   22    50
27  1610612753       11        25       10        26   21    51
28  1610612765       13        23        7        29   20    52
29  1610612745        9        27        8        28   17    55
```

In [162]:
def extract_season_standings():
    """
    >>> season_standings = extract_season_standings()
    >>> type(season_standings)
    pandas.core.frame.DataFrame
    >>> season_standings.shape
    (30, 7)
    >>> (season_standings['win'] == season_standings['homeWin'] + season_standings['awayWin']).sum()
    30
    >>> (season_standings['loss'] == season_standings['homeLoss'] + season_standings['awayLoss']).sum()
    30
    """
    ### BEGIN SOLUTION
    data = pd.read_json('standings_all.json')
    data = data['league']['standard']
    season_standings = pd.DataFrame.from_dict(data['teams'])
    col = ['teamId', 'homeWin', 'homeLoss', 'awayWin', 'awayLoss', 'win', 'loss']
    return season_standings[col].astype(int)
    ### END SOLUTION

## 01. Define a function named `find_most_win_by_division` that is able to find the most win team for each division given `standings_all.json` and `teams.json` as a pandas DataFrame.

- Expected inputs: None.
- Expected outputs: a (6, 4) DataFrame.

```
  confName    divName            fullName  win
0     East   Atlantic  Philadelphia 76ers   49
1     East    Central     Milwaukee Bucks   46
2     East  Southeast       Atlanta Hawks   41
3     West  Northwest           Utah Jazz   52
4     West    Pacific        Phoenix Suns   51
5     West  Southwest    Dallas Mavericks   42
```

In [225]:
def find_most_win_by_division():
    """
    >>> most_win_by_division = find_most_win_by_division()
    >>> type(most_win_by_division)
    pandas.core.frame.DataFrame
    >>> most_win_by_division.shape
    (6, 4)
    >>> most_win_by_division['win'].max()
    52
    >>> most_win_by_division['win'].min()
    41
    """
    ### BEGIN SOLUTION
    season_standings = extract_season_standings()
    season_standings
    season_standings = season_standings.astype(str)

    teams = pd.read_json('teams.json')
    teams = teams['league']['standard']
    teams_pd = pd.DataFrame.from_dict(teams)

    # Merge
    result = pd.merge(season_standings, teams_pd, on = 'teamId')
    max_win = result.groupby('divName')['win'].max()

    # Atlantic
    res = pd.DataFrame()
    res = res.append(result[result['win'] == max_win['Atlantic']])

    # Central
    cen = result[result['divName'] == 'Central']
    res = res.append(cen[cen['win'] == max_win['Central']])

    # Southeast
    se = result[result['divName'] == 'Southeast']
    res = res.append(se[se['win'] == max_win['Southeast']])

    # Northwest
    nw = result[result['divName'] == 'Northwest']
    res = res.append(nw[nw['win'] == max_win['Northwest']])

    # Pacific
    pf = result[result['divName'] == 'Pacific']
    res = res.append(pf[pf['win'] == max_win['Pacific']])

    # Southwest
    sw = result[result['divName'] == 'Southwest']
    res = res.append(sw[sw['win'] == max_win['Southwest']])
    col = ['confName', 'divName', 'fullName', 'win']
    res = res[col].reset_index()
    res = res.astype({'win': 'int64'})
    return res.drop(columns = 'index')    
    ### END SOLUTION

## 02. Define a function named `extract_nba_coaches` that is able to extract all coaches including head coaches and assistant coaches given `coaches.json` and `teams.json` as a pandas DataFrame.

- Expected inputs: None.
- Expected outputs: a (236, 4) DataFrame.

```
               fullName firstName   lastName  isAssistant
0    Philadelphia 76ers       Doc     Rivers        False
1    Philadelphia 76ers     David    Joerger         True
2    Philadelphia 76ers       Sam    Cassell         True
3    Philadelphia 76ers       Dan      Burke         True
4    Philadelphia 76ers    Popeye      Jones         True
..                  ...       ...        ...          ...
231  Washington Wizards     David     Adkins         True
232  Washington Wizards    Jarell  Christian         True
233  Washington Wizards      Dean     Oliver         True
234  Washington Wizards     Corey     Gaines         True
235  Washington Wizards      Mike   Terpstra         True

[236 rows x 4 columns]
```

In [255]:
def extract_nba_coaches():
    """
    >>> nba_coaches = extract_nba_coaches()
    >>> type(nba_coaches)
    pandas.core.frame.DataFrame
    >>> nba_coaches.shape
    (236, 4)
    >>> nba_coaches['fullName'].nunique()
    30
    >>> nba_coaches['isAssistant'].sum()
    210
    """
    ### BEGIN SOLUTION
    coaches = pd.read_json('coaches.json')
    coaches = coaches['league']['standard']
    coaches = pd.DataFrame.from_dict(coaches)

    teams = pd.read_json('teams.json')
    teams = teams['league']['standard']
    teams = pd.DataFrame.from_dict(teams)

    coaches = coaches.drop(columns = ['sortSequence', 'college', 'teamSitesOnly'])
    col = ['teamId', 'fullName']

    res = pd.merge(coaches, teams[col], on = 'teamId')
    col = ['fullName', 'firstName', 'lastName', 'isAssistant']
    return res[col]    
    ### END SOLUTION

## 03. Define a function named `calculate_confirmed_death_rate_by_countries` according to the following formula given `06-12-2021.csv` and `UID_ISO_FIPS_LookUp_Table.csv`.

$$
\text{Death Rate} = \frac{\text{Deaths}}{\text{Confirmed}} \\
\text{Confirmed Rate} = \frac{\text{Confirmed}}{\text{Population}}
$$

- Expected inputs: None.
- Expected outputs: a (193, 5) DataFrame.

```
                    Confirmed  Deaths  Population  Confirmed_Rate  Death_Rate
Country_Region                                                               
Afghanistan             88740    3449  38928341.0        0.002280    0.038866
Albania                132449    2453   2877800.0        0.046024    0.018520
Algeria                133070    3565  43851043.0        0.003035    0.026790
Andorra                 13813     127     77265.0        0.178774    0.009194
Angola                  36600     825  32866268.0        0.001114    0.022541
...                       ...     ...         ...             ...         ...
Vietnam                 10337      58  97338583.0        0.000106    0.005611
West Bank and Gaza     311018    3524   5101416.0        0.060967    0.011331
Yemen                    6857    1347  29825968.0        0.000230    0.196442
Zambia                 110332    1365  18383956.0        0.006002    0.012372
Zimbabwe                39852    1632  14862927.0        0.002681    0.040952

[193 rows x 5 columns]
```

In [274]:
def calculate_confirmed_death_rate_by_countries():
    """
    >>> confirmed_death_rate_by_countries = calculate_confirmed_death_rate_by_countries()
    >>> type(confirmed_death_rate_by_countries)
    pandas.core.frame.DataFrame
    >>> confirmed_death_rate_by_countries.shape
    (193, 5)
    >>> print(confirmed_death_rate_by_countries.loc[['Taiwan*'], :])
                    Confirmed  Deaths  Population  Confirmed_Rate  Death_Rate
    Country_Region                                                           
    Taiwan*             12746     411  23816775.0        0.000535    0.032245
    """
    ### BEGIN SOLUTION
    lookup_table = pd.read_csv('UID_ISO_FIPS_LookUp_Table.csv')
    daily_report = pd.read_csv('06-12-2021.csv')

    confirmed = daily_report.groupby('Country_Region')['Confirmed'].sum()
    deaths = daily_report.groupby('Country_Region')['Deaths'].sum()
    res = pd.merge(confirmed, deaths, on = 'Country_Region')

    population = lookup_table.groupby('Country_Region')['Population'].sum()

    res = pd.merge(res, population, on = 'Country_Region')
    res['Confirmed_Rate'] = res['Confirmed'] / res['Population']
    res['Death_Rate'] = res['Deaths'] / res['Population']

    return res
    ### END SOLUTION

## 04. Define a function named `calculate_daily_cases_of_taiwan` that is able to calculate the daily cases of Taiwan a DataFrame as expected given `time_series_covid19_confirmed_global.csv` and `time_series_covid19_deaths_global.csv`.

- Expected inputs: None.
- Expected outputs: a (508, 5) DataFrame.

```
           Country/Region  Confirmed  Deaths  Daily_Confirmed  Daily_Deaths
Date                                                                       
2020-01-22        Taiwan*          1       0              NaN           NaN
2020-01-23        Taiwan*          1       0              0.0           0.0
2020-01-24        Taiwan*          3       0              2.0           0.0
2020-01-25        Taiwan*          3       0              0.0           0.0
2020-01-26        Taiwan*          4       0              1.0           0.0
...                   ...        ...     ...              ...           ...
2021-06-08        Taiwan*      11694     308            203.0          22.0
2021-06-09        Taiwan*      11968     333            274.0          25.0
2021-06-10        Taiwan*      12222     361            254.0          28.0
2021-06-11        Taiwan*      12500     385            278.0          24.0
2021-06-12        Taiwan*      12746     411            246.0          26.0

[508 rows x 5 columns]
```

In [318]:
def calculate_daily_cases_of_taiwan():
    """
    >>> daily_cases_of_taiwan = calculate_daily_cases_of_taiwan()
    >>> type(daily_cases_of_taiwan)
    pandas.core.frame.DataFrame
    >>> daily_cases_of_taiwan.shape
    (508, 5)
    """
    ### BEGIN SOLUTION
    def change_date(d):
        month = d[0]
        date = d[1]
        year = '20'+ d[2]

        if len(date) < 2:
            date = '0' + date
        if len(month) < 2:
            month = '0'+ month

        return year + '-' + month + '-' + date

    confirmed_global = pd.read_csv('time_series_covid19_confirmed_global.csv')
    deaths_global = pd.read_csv('time_series_covid19_deaths_global.csv')

    taiwan_conf = confirmed_global[confirmed_global['Country/Region'] == 'Taiwan*']
    taiwan_conf = taiwan_conf.drop(columns = ['Province/State', 'Lat', 'Long'])

    taiwan_deaths = deaths_global[deaths_global['Country/Region'] == 'Taiwan*']
    taiwan_deaths = taiwan_deaths.drop(columns = ['Province/State', 'Lat', 'Long'])

    idVars = ['Country/Region']
    conf = pd.melt(taiwan_conf,
                   id_vars = idVars,
                   var_name='Date',
                   value_name='Confirmed')

    deaths = pd.melt(taiwan_deaths,
                   id_vars = idVars,
                   var_name='Date',
                   value_name='Deaths')

    res = pd.merge(conf, deaths, on = ['Date', 'Country/Region'])
    res['Date'] = res['Date'].str.split('/').map(change_date)

    res['Daily_Confirmed'] = 0.0
    res['Daily_Deaths'] = 0.0

    for i in range(1, res.shape[0]):
        comfirmed_plus = float(res.iloc[i, 2]- res.iloc[i-1, 2])
        deaths_plus = float(res.iloc[i, 3]- res.iloc[i-1, 3])
        res.iloc[i, 4] = comfirmed_plus
        res.iloc[i, 5] = deaths_plus

    res.iloc[0, 4] = np.nan
    res.iloc[0, 5] = np.nan

    return res.set_index('Date')
    ### END SOLUTION

## Run tests!

Kernel -> Restart & Run All. -> Restart And Run All Cells.

In [319]:
class TestFinal(unittest.TestCase):
    def test_00_extract_season_standings(self):
        season_standings = extract_season_standings()
        self.assertIsInstance(season_standings, pd.core.frame.DataFrame)
        self.assertEqual(season_standings.shape, (30, 7))
        self.assertEqual((season_standings['win'] == season_standings['homeWin'] + season_standings['awayWin']).sum(), 30)
        self.assertEqual((season_standings['loss'] == season_standings['homeLoss'] + season_standings['awayLoss']).sum(), 30)
    def test_01_find_most_win_by_division(self):
        most_win_by_division = find_most_win_by_division()
        self.assertIsInstance(most_win_by_division, pd.core.frame.DataFrame)
        self.assertEqual(most_win_by_division.shape, (6, 4))
        self.assertEqual(most_win_by_division['win'].max(), 52)
        self.assertEqual(most_win_by_division['win'].min(), 41)
    def test_02_extract_nba_coaches(self):
        nba_coaches = extract_nba_coaches()
        self.assertIsInstance(nba_coaches, pd.core.frame.DataFrame)
        self.assertEqual(nba_coaches.shape, (236, 4))
        self.assertEqual(nba_coaches['fullName'].nunique(), 30)
        self.assertEqual(nba_coaches['isAssistant'].sum(), 210)
    def test_03_calculate_confirmed_death_rate_by_countries(self):
        confirmed_death_rate_by_countries = calculate_confirmed_death_rate_by_countries()
        self.assertIsInstance(confirmed_death_rate_by_countries, pd.core.frame.DataFrame)
        self.assertEqual(confirmed_death_rate_by_countries.shape, (193, 5))
        tw = confirmed_death_rate_by_countries.loc[['Taiwan*'], :]
        self.assertEqual(tw['Confirmed'].values[0], 12746)
        self.assertEqual(tw['Deaths'].values[0], 411)
        self.assertAlmostEqual(tw['Population'].values[0], 23816775.0)
    def test_04_calculate_daily_cases_of_taiwan(self):
        daily_cases_of_taiwan = calculate_daily_cases_of_taiwan()
        self.assertIsInstance(daily_cases_of_taiwan, pd.core.frame.DataFrame)
        self.assertEqual(daily_cases_of_taiwan.shape, (508, 5))
        self.assertEqual(daily_cases_of_taiwan['Country/Region'].unique()[0], 'Taiwan*')
        self.assertTrue(np.isnan(daily_cases_of_taiwan['Daily_Confirmed'].values[0]), True)
        self.assertTrue(np.isnan(daily_cases_of_taiwan['Daily_Deaths'].values[0]), True)
        self.assertAlmostEqual(daily_cases_of_taiwan['Daily_Confirmed'].values[-1], 246.0)
        self.assertAlmostEqual(daily_cases_of_taiwan['Daily_Deaths'].values[-1], 26.0)

suite = unittest.TestLoader().loadTestsFromTestCase(TestFinal)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)

test_00_extract_season_standings (__main__.TestFinal) ... ok
test_01_find_most_win_by_division (__main__.TestFinal) ... ok
test_02_extract_nba_coaches (__main__.TestFinal) ... ok
test_03_calculate_confirmed_death_rate_by_countries (__main__.TestFinal) ... ok
test_04_calculate_daily_cases_of_taiwan (__main__.TestFinal) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.629s

OK


In [320]:
print("You've got {} points among {} questions.".format(number_of_successes * 5, number_of_test_runs))

You've got 25 points among 5 questions.
